# Importing required libraries.

In [101]:
from os import listdir
import pandas as pd
import math
from time import time
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

# Preprocessing CSV dataset to dictionaries of dfs.

In [8]:
filedir = "/content/drive/MyDrive/all_data/"
datasets = defaultdict(lambda: defaultdict(lambda: defaultdict()))
for filename in listdir(filedir):
  typ, clauses, size = filename.split('_')
  datasets[int(clauses[1:])][int(size[1:-4])][typ] = pd.read_csv(filedir+"/"+filename, sep = ',', header = None)

# Decision Tree Classifier

In [123]:
for num_clauses in [300, 500, 1000, 1500, 1800]:
  for data_size in [100, 1000, 5000]:
    st = time()
    trainX, trainy = datasets[num_clauses][data_size]['train'].iloc[:,:-1], datasets[num_clauses][data_size]['train'].iloc[:,-1]
    validX, validy = datasets[num_clauses][data_size]['valid'].iloc[:,:-1], datasets[num_clauses][data_size]['valid'].iloc[:,-1]
    
    print('===========================================================================')
    print('c',num_clauses,'d',data_size)
    print('===========================================================================')

    print("Finding optimal parameters:")
    #Optimizing on depth
    store_accs2 = []
    with tqdm(total=3, position=0, leave=True) as pbar:
      for attr in ['sqrt', 'log2', None]:
        for try_reduce_depth in range(data_size//50, int(data_size//2),data_size//50):
          clf = DecisionTreeClassifier(random_state = 0, max_features=attr, min_samples_leaf=try_reduce_depth).fit(trainX, trainy)
          tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
          store_accs2.append((attr, try_reduce_depth, val_acc))
        pbar.update()
    optimal_num_feat, optimal_depth_red, val_acc_best= max(store_accs2, key = lambda x: x[2])
    print("\nValidation Accuracy:", val_acc_best, "For", optimal_num_feat, "max_features and", optimal_depth_red, "min_samples_leaf")

    #Combined train and validation dataset
    fin_trainX, fin_trainy = pd.concat([trainX, validX], axis = 0), pd.concat([trainy, validy], axis = 0)
    clf = DecisionTreeClassifier(random_state = 0, max_features=optimal_num_feat, min_samples_leaf=optimal_depth_red).fit(fin_trainX, fin_trainy)

    testX, testy = datasets[num_clauses][data_size]['test'].iloc[:,:-1], datasets[num_clauses][data_size]['test'].iloc[:,-1]
    ypred = clf.predict(testX)
    test_acc = clf.score(testX, testy) 
    precision, recall, f1, _ = precision_recall_fscore_support(testy, ypred, average = 'macro')
    print("\nOn test dataset:\n", "Accuracy:", test_acc, "\n", "Precision:", precision, "\n", "Recall:", recall, "\n", "F1:", f1, "\n")
    print("Total Time Taken:", time()-st,"\n")

c 300 d 100
Finding optimal parameters:


100%|██████████| 3/3 [00:01<00:00,  2.37it/s]



Validation Accuracy: 0.65 For None max_features and 12 min_samples_leaf

On test dataset:
 Accuracy: 0.63 
 Precision: 0.6302083333333333 
 Recall: 0.63 
 F1: 0.6298519407763106 

Total Time Taken: 1.3187751770019531 

c 300 d 1000
Finding optimal parameters:


100%|██████████| 3/3 [00:02<00:00,  1.25it/s]



Validation Accuracy: 0.6485 For None max_features and 80 min_samples_leaf

On test dataset:
 Accuracy: 0.647 
 Precision: 0.6480446027167694 
 Recall: 0.647 
 F1: 0.6463762076302597 

Total Time Taken: 2.601274251937866 

c 300 d 5000
Finding optimal parameters:


...


Validation Accuracy: 0.98 For None max_features and 6 min_samples_leaf

On test dataset:
 Accuracy: 0.94 
 Precision: 0.9401760704281713 
 Recall: 0.94 
 F1: 0.93999399939994 

Total Time Taken: 1.248870849609375 

c 1800 d 1000
Finding optimal parameters:


100%|██████████| 3/3 [00:02<00:00,  1.24it/s]



Validation Accuracy: 0.962 For None max_features and 20 min_samples_leaf

On test dataset:
 Accuracy: 0.9595 
 Precision: 0.961105106877039 
 Recall: 0.9595 
 F1: 0.9594647241762143 

Total Time Taken: 2.6088056564331055 

c 1800 d 5000
Finding optimal parameters:


100%|██████████| 3/3 [00:09<00:00,  3.15s/it]



Validation Accuracy: 0.9694 For None max_features and 100 min_samples_leaf

On test dataset:
 Accuracy: 0.9681 
 Precision: 0.9685502768160201 
 Recall: 0.9681 
 F1: 0.9680923341832876 

Total Time Taken: 10.467654943466187 



# Bagging Classifier

In [122]:
for num_clauses in [300, 500, 1000, 1500, 1800]:
  for data_size in [100, 1000, 5000]:
    st = time()
    trainX, trainy = datasets[num_clauses][data_size]['train'].iloc[:,:-1], datasets[num_clauses][data_size]['train'].iloc[:,-1]
    validX, validy = datasets[num_clauses][data_size]['valid'].iloc[:,:-1], datasets[num_clauses][data_size]['valid'].iloc[:,-1] 
    
    print('===========================================================================')
    print('c',num_clauses,'d',data_size)
    print('===========================================================================')

    print("Finding optimal parameters:")
    #Optimizing on depth
    store_accs2 = []
    with tqdm(total=3, position=0, leave=True) as pbar:
      for name_try, attr in [('None', trainX.shape[1]),('sqrt', int(math.sqrt(trainX.shape[1]))), ('log2', int(math.log2(trainX.shape[1])))]:
        for try_estimator in range(15, 30  ,5):
          clf = BaggingClassifier(random_state = 0, estimator=DecisionTreeClassifier(), max_features=attr, n_estimators=try_estimator).fit(trainX, trainy)
          tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
          store_accs2.append((attr, try_estimator, val_acc))
        pbar.update()
    optimal_num_feat, optimal_est_n, val_acc_best= max(store_accs2, key = lambda x: x[2])
    print("\nValidation Accuracy:", val_acc_best, "For", optimal_num_feat, "max_features and", optimal_est_n, "n_estimators")

    #Combined train and validation dataset
    fin_trainX, fin_trainy = pd.concat([trainX, validX], axis = 0), pd.concat([trainy, validy], axis = 0)
    clf = BaggingClassifier(random_state = 0, estimator=DecisionTreeClassifier(), max_features=optimal_num_feat, n_estimators=optimal_est_n).fit(fin_trainX, fin_trainy)

    testX, testy = datasets[num_clauses][data_size]['test'].iloc[:,:-1], datasets[num_clauses][data_size]['test'].iloc[:,-1]
    ypred = clf.predict(testX)
    test_acc = clf.score(testX, testy) 
    precision, recall, f1, _ = precision_recall_fscore_support(testy, ypred, average = 'macro')
    print("\nOn test dataset:\n", "Accuracy:", test_acc, "\n", "Precision:", precision, "\n", "Recall:", recall, "\n", "F1:", f1, "\n")
    print("Total Time Taken:", time()-st,"\n")

c 300 d 100
Finding optimal parameters:


100%|██████████| 3/3 [00:02<00:00,  1.37it/s]



Validation Accuracy: 0.63 For 500 max_features and 15 n_estimators

On test dataset:
 Accuracy: 0.675 
 Precision: 0.675017501750175 
 Recall: 0.675 
 F1: 0.67499187479687 

Total Time Taken: 2.5724146366119385 

c 300 d 1000
Finding optimal parameters:


100%|██████████| 3/3 [00:06<00:00,  2.08s/it]



Validation Accuracy: 0.804 For 500 max_features and 25 n_estimators

On test dataset:
 Accuracy: 0.8225 
 Precision: 0.8226422852477943 
 Recall: 0.8225 
 F1: 0.8224804284672386 

Total Time Taken: 10.36277461051941 

c 300 d 5000
Finding optimal parameters:


...


Validation Accuracy: 0.995 For 500 max_features and 20 n_estimators

On test dataset:
 Accuracy: 0.97 
 Precision: 0.970188075230092 
 Recall: 0.97 
 F1: 0.96999699969997 

Total Time Taken: 1.0107195377349854 

c 1800 d 1000
Finding optimal parameters:


100%|██████████| 3/3 [00:05<00:00,  1.69s/it]



Validation Accuracy: 1.0 For 22 max_features and 25 n_estimators

On test dataset:
 Accuracy: 1.0 
 Precision: 1.0 
 Recall: 1.0 
 F1: 1.0 

Total Time Taken: 5.349790096282959 

c 1800 d 5000
Finding optimal parameters:


100%|██████████| 3/3 [00:21<00:00,  7.27s/it]



Validation Accuracy: 0.9998 For 8 max_features and 25 n_estimators

On test dataset:
 Accuracy: 0.9997 
 Precision: 0.9997000199880008 
 Recall: 0.9997 
 F1: 0.999699999997 

Total Time Taken: 22.264930248260498 



# Random Forest Classifier

In [94]:
for num_clauses in [300, 500, 1000, 1500, 1800]:
  for data_size in [100, 1000, 5000]:
    st = time()
    trainX, trainy = datasets[num_clauses][data_size]['train'].iloc[:,:-1], datasets[num_clauses][data_size]['train'].iloc[:,-1]
    validX, validy = datasets[num_clauses][data_size]['valid'].iloc[:,:-1], datasets[num_clauses][data_size]['valid'].iloc[:,-1]
    clf = RandomForestClassifier(max_features=trainX.shape[1], min_samples_leaf=1).fit(trainX, trainy)
    tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
    
    print('===========================================================================')
    print('c',num_clauses,'d',data_size)
    print('===========================================================================')

    print("Finding optimal parameters:")
    store_accs2 = []
    with tqdm(total=3, position=0, leave=True) as pbar:
      for attr in ['sqrt', 'log2', None]:
        for try_reduce_depth in range(data_size//50, int(data_size*val_acc_best),data_size//50):
          clf = RandomForestClassifier(random_state = 0, max_features=attr, min_samples_leaf=try_reduce_depth).fit(trainX, trainy)
          tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
          store_accs2.append((attr, try_reduce_depth, val_acc))
        pbar.update()
    optimal_num_feat, optimal_depth_red, val_acc_best= max(store_accs2, key = lambda x: x[2])
    print("\nValidation Accuracy:", val_acc_best, "For", optimal_num_feat, "max_features and", optimal_depth_red, "min_samples_leaf")

    #Combined train and validation dataset
    fin_trainX, fin_trainy = pd.concat([trainX, validX], axis = 0), pd.concat([trainy, validy], axis = 0)
    clf = RandomForestClassifier(random_state = 0, max_features=optimal_num_feat, min_samples_leaf=optimal_depth_red).fit(fin_trainX, fin_trainy)

    testX, testy = datasets[num_clauses][data_size]['test'].iloc[:,:-1], datasets[num_clauses][data_size]['test'].iloc[:,-1]
    ypred = clf.predict(testX)
    test_acc = clf.score(testX, testy) 
    precision, recall, f1, _ = precision_recall_fscore_support(testy, ypred, average = 'macro')
    print("\nOn test dataset:\n", "Accuracy:", test_acc, "\n", "Precision:", precision, "\n", "Recall:", recall, "\n", "F1:", f1, "\n")
    print("Total Time Taken:", time()-st,"\n")

c 300 d 100
Finding optimal parameters:


100%|██████████| 3/3 [00:36<00:00, 12.20s/it]



Validation Accuracy: 0.73 For sqrt max_features and 10 min_samples_leaf

On test dataset:
 Accuracy: 0.77 
 Precision: 0.7717391304347826 
 Recall: 0.77 
 F1: 0.7696314102564102 

Total Time Taken: 37.854525566101074 

c 300 d 1000
Finding optimal parameters:


100%|██████████| 3/3 [01:49<00:00, 36.44s/it]



Validation Accuracy: 0.8705 For sqrt max_features and 20 min_samples_leaf

On test dataset:
 Accuracy: 0.8725 
 Precision: 0.8725033525301727 
 Recall: 0.8725 
 F1: 0.8724997131243546 

Total Time Taken: 119.38709926605225 

c 300 d 5000
Finding optimal parameters:


...


Validation Accuracy: 1.0 For sqrt max_features and 2 min_samples_leaf

On test dataset:
 Accuracy: 1.0 
 Precision: 1.0 
 Recall: 1.0 
 F1: 1.0 

Total Time Taken: 49.158920764923096 

c 1800 d 1000
Finding optimal parameters:


100%|██████████| 3/3 [01:36<00:00, 32.12s/it]



Validation Accuracy: 1.0 For sqrt max_features and 20 min_samples_leaf

On test dataset:
 Accuracy: 1.0 
 Precision: 1.0 
 Recall: 1.0 
 F1: 1.0 

Total Time Taken: 102.43275570869446 

c 1800 d 5000
Finding optimal parameters:


100%|██████████| 3/3 [06:15<00:00, 125.22s/it]



Validation Accuracy: 1.0 For sqrt max_features and 1000 min_samples_leaf

On test dataset:
 Accuracy: 0.9999 
 Precision: 0.9999000199960009 
 Recall: 0.9999 
 F1: 0.9998999999989999 

Total Time Taken: 407.1577808856964 



# Gradient Boosting Classifier

In [99]:
for num_clauses in [300, 500, 1000, 1500, 1800]:
  for data_size in [100, 1000, 5000]:
    st = time()
    trainX, trainy = datasets[num_clauses][data_size]['train'].iloc[:,:-1], datasets[num_clauses][data_size]['train'].iloc[:,-1]
    validX, validy = datasets[num_clauses][data_size]['valid'].iloc[:,:-1], datasets[num_clauses][data_size]['valid'].iloc[:,-1]
    
    print('===========================================================================')
    print('c',num_clauses,'d',data_size)
    print('===========================================================================')

    print("Finding optimal n_estimators:")
    #Optimizing on depth
    store_accs2 = []
    with tqdm(total=3, position=0, leave=True) as pbar:
      for attr in ['sqrt', 'log2', None]:
        for try_estimator in range(15, 45, 5):
          clf = GradientBoostingClassifier(random_state = 0, max_features=attr, n_estimators=try_estimator).fit(trainX, trainy)
          tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
          store_accs2.append((attr, try_estimator, val_acc))
        pbar.update()
    optimal_num_feat, optimal_est_n, val_acc_best= max(store_accs2, key = lambda x: x[2])

    print("Finding optimal learning rate:")
    store_accs3 = [(0.1, val_acc_best)]
    with tqdm(total=3, position=0, leave=True) as pbar:
      for lr in [0.01, 0.5, 1]:
        clf = GradientBoostingClassifier(random_state = 0, max_features=optimal_num_feat, n_estimators=optimal_est_n, learning_rate = lr).fit(trainX, trainy)
        tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
        store_accs3.append((lr,val_acc))
        pbar.update()
    opt_lr, val_acc_best= max(store_accs3, key = lambda x: x[1])
    print("\nValidation Accuracy:", val_acc_best, "For", optimal_num_feat, "max_features,", optimal_est_n, "n_estimators and", opt_lr, "learning rate.")

    #Combined train and validation dataset
    fin_trainX, fin_trainy = pd.concat([trainX, validX], axis = 0), pd.concat([trainy, validy], axis = 0)
    clf = GradientBoostingClassifier(random_state = 0, max_features=optimal_num_feat, n_estimators=optimal_est_n, learning_rate = opt_lr).fit(fin_trainX, fin_trainy)

    testX, testy = datasets[num_clauses][data_size]['test'].iloc[:,:-1], datasets[num_clauses][data_size]['test'].iloc[:,-1]
    ypred = clf.predict(testX)
    test_acc = clf.score(testX, testy) 
    precision, recall, f1, _ = precision_recall_fscore_support(testy, ypred, average = 'macro')
    print("\nOn test dataset:\n", "Accuracy:", test_acc, "\n", "Precision:", precision, "\n", "Recall:", recall, "\n", "F1:", f1, "\n")
    print("Total Time Taken:", time()-st,"\n")

c 300 d 100
Finding optimal n_estimators:


100%|██████████| 3/3 [00:03<00:00,  1.13s/it]


Finding optimal learning rate:


100%|██████████| 3/3 [00:01<00:00,  1.93it/s]



Validation Accuracy: 0.715 For None max_features, 30 n_estimators and 0.1 learning rate.

On test dataset:
 Accuracy: 0.83 
 Precision: 0.8311922922521076 
 Recall: 0.8300000000000001 
 F1: 0.8298468621759585 

Total Time Taken: 5.566346645355225 

c 300 d 1000
Finding optimal n_estimators:


...


Validation Accuracy: 1.0 For sqrt max_features, 20 n_estimators and 0.1 learning rate.

On test dataset:
 Accuracy: 0.998 
 Precision: 0.99800796812749 
 Recall: 0.998 
 F1: 0.997999991999968 

Total Time Taken: 7.411762475967407 

c 1800 d 5000
Finding optimal n_estimators:


100%|██████████| 3/3 [00:36<00:00, 12.03s/it]


Finding optimal learning rate:


100%|██████████| 3/3 [00:00<00:00,  3.12it/s]



Validation Accuracy: 1.0 For log2 max_features, 40 n_estimators and 0.5 learning rate.

On test dataset:
 Accuracy: 1.0 
 Precision: 1.0 
 Recall: 1.0 
 F1: 1.0 

Total Time Taken: 37.634262800216675 



# Downloading MNIST dataset.

In [102]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X / 255.
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]
data_size = 60000
trainX, validX, trainy, validy = train_test_split(X_train, y_train, test_size=0.33, random_state=0)

/usr/local/lib/python3.9/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


# Using Decision Tree Classifier on MNIST

In [108]:
print("Finding optimal parameters:")
#Optimizing on depth
store_accs2 = []
with tqdm(total=3, position=0, leave=True) as pbar:
  for attr in ['sqrt', 'log2', None]:
    for try_reduce_depth in [1] + list(range(data_size//50, data_size//2,data_size//50)):
      clf = DecisionTreeClassifier(random_state = 0, max_features=attr, min_samples_leaf=try_reduce_depth).fit(trainX, trainy)
      tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
      store_accs2.append((attr, try_reduce_depth, val_acc))
    pbar.update()
optimal_num_feat, optimal_depth_red, val_acc_best= max(store_accs2, key = lambda x: x[2])
print("\nValidation Accuracy:", val_acc_best, "For", optimal_num_feat, "max_features and", optimal_depth_red, "min_samples_leaf")

#Combined train and validation dataset
clf = DecisionTreeClassifier(random_state = 0, max_features=optimal_num_feat, min_samples_leaf=optimal_depth_red).fit(X_train, y_train)
ypred = clf.predict(X_test)
test_acc = clf.score(X_test, y_test) 
precision, recall, f1, _ = precision_recall_fscore_support(y_test, ypred, average = 'macro')
print("\nOn test dataset:\n", "Accuracy:", test_acc, "\n", "Precision:", precision, "\n", "Recall:", recall, "\n", "F1:", f1, "\n")
print("Total Time Taken:", time()-st,"\n")

Finding optimal parameters:


100%|██████████| 3/3 [01:23<00:00, 27.86s/it]



Validation Accuracy: 0.8625757575757576 For None max_features and 1 min_samples_leaf

On test dataset:
 Accuracy: 0.8779 
 Precision: 0.8766071633824115 
 Recall: 0.8763956057044318 
 F1: 0.8764276413789659 

Total Time Taken: 1464.451583623886 



# Using Bagging Classifier on MNIST

In [119]:
print("Finding optimal parameters:")
#Optimizing on depth
store_accs2 = []
with tqdm(total=2, position=0, leave=True) as pbar:
  for name_try, attr in [('sqrt', int(math.sqrt(trainX.shape[1]))), ('log2', int(math.log2(trainX.shape[1])))]:
    for try_estimator in range(10, 25, 5):
      clf = BaggingClassifier(random_state = 0, estimator=DecisionTreeClassifier(), max_features=attr, n_estimators=try_estimator).fit(trainX, trainy)
      tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
      store_accs2.append((try_max_features, try_estimator, val_acc))
    pbar.update()
optimal_num_feat, optimal_est_n, val_acc_best= max(store_accs2, key = lambda x: x[2])
print("\nValidation Accuracy:", val_acc_best, "For", optimal_num_feat, "max_features and", optimal_est_n, "n_estimators")

#Combined train and validation dataset
clf = BaggingClassifier(random_state = 0, estimator=DecisionTreeClassifier(), max_features=optimal_num_feat, n_estimators=optimal_est_n).fit(X_train, y_train)

ypred = clf.predict(X_test)
test_acc = clf.score(X_test, y_test) 
precision, recall, f1, _ = precision_recall_fscore_support(y_test, ypred, average = 'macro')
print("\nOn test dataset:\n", "Accuracy:", test_acc, "\n", "Precision:", precision, "\n", "Recall:", recall, "\n", "F1:", f1, "\n")
print("Total Time Taken:", time()-st,"\n")

Finding optimal parameters:


100%|██████████| 2/2 [00:19<00:00,  9.58s/it]



Validation Accuracy: 0.8997474747474747 For 499 max_features and 20 n_estimators

On test dataset:
 Accuracy: 0.9582 
 Precision: 0.9579025278375541 
 Recall: 0.9577075481220423 
 F1: 0.9577531082748332 

Total Time Taken: 3540.0547671318054 



# Using Random Forest Classifier on MNIST

In [115]:
print("Finding optimal parameters:")
store_accs2 = []
with tqdm(total=2, position=0, leave=True) as pbar:
  for attr in ['sqrt', 'log2']:
    for try_reduce_depth in [1]:
      clf = RandomForestClassifier(random_state = 0, max_features=attr, min_samples_leaf=try_reduce_depth).fit(trainX, trainy)
      tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
      store_accs2.append((attr, try_reduce_depth, val_acc))
    pbar.update()
optimal_num_feat, optimal_depth_red, val_acc_best= max(store_accs2, key = lambda x: x[2])
print("\nValidation Accuracy:", val_acc_best, "For", optimal_num_feat, "max_features and", optimal_depth_red, "min_samples_leaf")

#Combined train and validation dataset
# fin_trainX, fin_trainy = pd.concat([trainX, validX], axis = 0), pd.concat([trainy, validy], axis = 0)
clf = RandomForestClassifier(random_state = 0, max_features=optimal_num_feat, min_samples_leaf=optimal_depth_red).fit(X_train, y_train)

ypred = clf.predict(X_test)
test_acc = clf.score(X_test, y_test) 
precision, recall, f1, _ = precision_recall_fscore_support(y_test, ypred, average = 'macro')
print("\nOn test dataset:\n", "Accuracy:", test_acc, "\n", "Precision:", precision, "\n", "Recall:", recall, "\n", "F1:", f1, "\n")
print("Total Time Taken:", time()-st,"\n")

Finding optimal parameters:


100%|██████████| 2/2 [01:10<00:00, 35.31s/it]



Validation Accuracy: 0.9675757575757575 For sqrt max_features and 1 min_samples_leaf

On test dataset:
 Accuracy: 0.9705 
 Precision: 0.9702292282943962 
 Recall: 0.9702805962559312 
 F1: 0.9702351062124602 

Total Time Taken: 3090.2914838790894 



# Using Gradient Boosting Classifier on MNIST

In [120]:
print("Finding optimal n_estimators:")
#Optimizing on depth
store_accs2 = []
with tqdm(total=2, position=0, leave=True) as pbar:
  for attr in ['sqrt', 'log2']:
    for try_estimator in range(10, 25, 5):
      clf = GradientBoostingClassifier(random_state = 0, max_features=attr, n_estimators=try_estimator).fit(trainX, trainy)
      tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
      store_accs2.append((attr, try_estimator, val_acc))
    pbar.update()
optimal_num_feat, optimal_est_n, val_acc_best= max(store_accs2, key = lambda x: x[2])

print("Finding optimal learning rate:")
store_accs3 = [(0.1, val_acc_best)]
with tqdm(total=3, position=0, leave=True) as pbar:
  for lr in [0.01, 0.5, 1]:
    clf = GradientBoostingClassifier(random_state = 0, max_features=optimal_num_feat, n_estimators=optimal_est_n, learning_rate = lr).fit(trainX, trainy)
    tr_acc, val_acc = clf.score(trainX, trainy), clf.score(validX, validy) 
    store_accs3.append((lr,val_acc))
    pbar.update()
opt_lr, val_acc_best= max(store_accs3, key = lambda x: x[1])
print("\nValidation Accuracy:", val_acc_best, "For", optimal_num_feat, "max_features,", optimal_est_n, "n_estimators and", opt_lr, "learning rate.")

#Combined train and validation dataset
clf = GradientBoostingClassifier(random_state = 0, max_features=optimal_num_feat, n_estimators=optimal_est_n, learning_rate = opt_lr).fit(X_train, y_train)

ypred = clf.predict(X_test)
test_acc = clf.score(X_test, y_test) 
precision, recall, f1, _ = precision_recall_fscore_support(y_test, ypred, average = 'macro')
print("\nOn test dataset:\n", "Accuracy:", test_acc, "\n", "Precision:", precision, "\n", "Recall:", recall, "\n", "F1:", f1, "\n")
print("Total Time Taken:", time()-st,"\n")

Finding optimal n_estimators:


100%|██████████| 2/2 [01:32<00:00, 46.36s/it]


Finding optimal learning rate:


100%|██████████| 3/3 [00:55<00:00, 18.44s/it]



Validation Accuracy: 0.9087373737373737 For sqrt max_features, 20 n_estimators and 0.5 learning rate.

On test dataset:
 Accuracy: 0.9143 
 Precision: 0.9136284397631893 
 Recall: 0.9130241144917202 
 F1: 0.9131369254092189 

Total Time Taken: 3766.884696483612 

